<a href="https://colab.research.google.com/github/M-Yaseen1325/AI-Fundamentals/blob/main/Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


necessary libraries

In [ ]:
# Install necessary libraries
!pip install mat73

import mat73
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


**Define dataset paths**


In [ ]:
train_dataset_path = '/content/drive/MyDrive/train_32x32.mat'
test_dataset_path = '/content/drive/MyDrive/test_32x32.mat'


 **Load data function**

In [ ]:
def load_data(path):
    """ Helper function for loading a MAT-File"""
    data = loadmat(path)  # Since the original dataset isn't using mat73 but loadmat
    return data['X'], data['y']


**Load training and test data**

In [ ]:
X_train, y_train = load_data(train_dataset_path)
X_test, y_test = load_data(test_dataset_path)


Inspect the shape


In [ ]:
print("Training Set", X_train.shape, y_train.shape)
print("Test Set", X_test.shape, y_test.shape)

Training Set (73257, 32, 32, 3) (73257, 1)
Test Set (26032, 32, 32, 3) (26032, 1)


**Transpose the dataset: (width, height, channels, samples) **

In [ ]:
X_train = np.transpose(X_train, (3, 0, 1, 2))
X_test = np.transpose(X_test, (3, 0, 1, 2))

print("Reshaped Training Set", X_train.shape, y_train.shape)
print("Reshaped Test Set", X_test.shape, y_test.shape)

Reshaped Training Set (73257, 32, 32, 3) (73257, 1)
Reshaped Test Set (26032, 32, 32, 3) (26032, 1)


 Normalize the image data

In [ ]:
X_train = X_train.astype('float32')/ 255.0
X_test = X_test.astype('float32')/ 255.0

Handle label 10

In [ ]:
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

Convert the labels to one-hot encoding


In [ ]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Split the training data into training and validation sets


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print("Training Set", X_train.shape, y_train.shape)
print("Validation Set", X_val.shape, y_val.shape)
print("Test Set", X_test.shape, y_test.shape)

Training Set (58605, 32, 32, 3) (58605, 10)
Validation Set (14652, 32, 32, 3) (14652, 10)
Test Set (26032, 32, 32, 3) (26032, 10)


 Building the CNN Model


In [ ]:
def build_model():
    model = Sequential()

    # First convolution layer
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Second convolution layer
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Third convolution layer
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Flatten the output
    model.add(Flatten())

    # Fully connected layer
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
     # Output layer (10 digits, softmax activation for classification)
    model.add(Dense(10, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    return model


Build the model


In [ ]:
model = build_model()

# Print model summary
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 30, 30, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 15, 15, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 13, 13, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 6, 6, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 4, 4, 128)           │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 2, 2, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          65,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 160,202 (625.79 KB)

 Trainable params: 160,202 (625.79 KB)

 Non-trainable params: 0 (0.00 B)

Training the Model


In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_val, y_val))

Epoch 1/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 88s 94ms/step - accuracy: 0.3587 - loss: 2.8323 - val_accuracy: 0.8048 - val_loss: 0.6626
Epoch 2/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 137s 89ms/step - accuracy: 0.7656 - loss: 0.7752 - val_accuracy: 0.8438 - val_loss: 0.5389
Epoch 3/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 80s 87ms/step - accuracy: 0.8170 - loss: 0.6238 - val_accuracy: 0.8557 - val_loss: 0.4858
Epoch 4/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 83s 88ms/step - accuracy: 0.8394 - loss: 0.5469 - val_accuracy: 0.8669 - val_loss: 0.4526
Epoch 5/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 79s 85ms/step - accuracy: 0.8528 - loss: 0.5006 - val_accuracy: 0.8765 - val_loss: 0.4247
Epoch 6/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 79s 86ms/step - accuracy: 0.8594 - loss: 0.4771 - val_accuracy: 0.8699 - val_loss: 0.4407
Epoch 7/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 82s 87ms/step - accuracy: 0.8699 - loss: 0.4471 - val_accuracy: 0.8842 - val_loss: 0.4027
Epoch 8/20
916/916 ━━━━━━━━━━━━━━━━━━━━ 78s 85ms/step - accuracy: 0.8744 - loss: 0.4196 -

 Evaluating the Model


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc * 100:.2f}%')

814/814 ━━━━━━━━━━━━━━━━━━━━ 21s 26ms/step - accuracy: 0.8913 - loss: 0.4812
Test accuracy: 89.31%
